In [ ]:
#import libraries
from datetime import datetime, timedelta
from shapely.geometry import Point, Polygon
import pandas as pd
import csv
import numpy as np
import seaborn as sns
from sklearn import preprocessing, svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# import dataset
northern_gannet_df = pd.read_csv("https://github.com/akarighattam/Pelagic-bird-project/blob/main/datasets/noga_obsv_200601-202312_masscoast_50.csv", sep='\t', index_col=["Unnamed: 0"])
weather_df = pd.read_csv("https://github.com/akarighattam/Pelagic-bird-project/blob/main/datasets/noga_weather_200601-202312_masscoast_50_000-012_3.csv", sep='\t', index_col=["Unnamed: 0"])

In [ ]:
weather_df.shape

In [ ]:
northern_gannet_df.head(10)

### Data Preprocessing

In [ ]:
# take the log of each count to help the model understand the correlation better (the data is exponential)
observation_column=northern_gannet_df['OBSERVATION COUNT'].tolist()
# since log is not defined for x=0, we can convert the counts to log(x+1)
northern_gannet_df['OBSERVATION COUNT']=pd.Series([np.log(x+1) for x in observation_column])

In [ ]:
noga_log=northern_gannet_df

In [ ]:
# filtering pelagic bird counts data to a coastal region of Rockport 
noga_log=noga_log[noga_log['LONGITUDE']>=-70.64112479594434]
noga_log=noga_log[noga_log['LATITUDE']>=42.68138642588193]
noga_log=noga_log[noga_log['LONGITUDE']<=-70.61618444441049]
noga_log=noga_log[noga_log['LATITUDE']<=42.69585507137888]

In [ ]:
# reset the index of the dataframe
indices=noga_log.index
noga_log=noga_log.reset_index(drop=True)
# only keep indices that are also in noga_largest_count
weather=weather_df.loc[indices].reset_index(drop=True)
# create new series with just the observation count
northern_gannet=noga_log['OBSERVATION COUNT']
weather_complete=pd.concat([weather, noga_log[['LATITUDE', 'LONGITUDE', 'days_cos', 'days_sin']]], axis=1)

In [ ]:
northern_gannet.shape

In [ ]:
weather_complete.head(10)

In [ ]:
# plot NOGA counts vs weather parameters
counts_df=pd.concat([northern_gannet,weather_complete[['windy009']]], axis=1)
counts_df=counts_df.rename(columns={'OBSERVATION COUNT':'observation count','windy009':'W <---      wind-y component at 9AM      ---> E'})
counts=sns.scatterplot(data=counts_df, x='W <---      wind-y component at 9AM      ---> E', y='observation count')

### Train and test arrays

In [ ]:
# complete weather dataset
weather_windxy_cos_sin_1=weather_complete[weather_complete.columns[-14:-4]]
weather_windxy_cos_sin_2=weather_complete[weather_complete.columns[-2:]]
weather_windxy_cos_sin=pd.concat([weather_windxy_cos_sin_1, weather_windxy_cos_sin_2], axis=1)
weather_windxy_cos_sin.head(10)

In [ ]:
# weather dataset for 6AM and 9AM
weather_0609_cos_sin_1=weather_windxy_cos_sin[weather_windxy_cos_sin.columns[2:4]]
weather_0609_cos_sin_2=weather_windxy_cos_sin[weather_windxy_cos_sin.columns[7:9]]
weather_0609_cos_sin_3=weather_windxy_cos_sin[weather_windxy_cos_sin.columns[-2:]]
weather_0609_cos_sin=pd.concat([weather_0609_cos_sin_1,weather_0609_cos_sin_2, weather_0609_cos_sin_3], axis=1)
weather_0609_cos_sin.head(10)

In [ ]:
X=weather_0609_cos_sin.to_numpy()
y=northern_gannet.to_numpy()

In [ ]:
print(X)

In [ ]:
# normalize X values using StandardScaler() formula: z=(x-u)/s
# wind_X
windx_avg=np.mean(X[:, 0:2])
windx_std=np.std(X[:, 0:2])
X[:, 0:2]-=windx_avg
X[:, 0:2]=np.divide(X[:, 0:2], windx_std)
# wind_y
windy_avg=np.mean(X[:, 2:4])
windy_std=np.std(X[:, 2:4])
X[:, 2:4]-=windy_avg
X[:, 2:4]=np.divide(X[:, 2:4], windy_std)
# normalize y values using MinMaxScaler() formula: z=(y-min)/(max-min)
counts_min=np.min(y)
counts_max=np.max(y)
y-=counts_min
y=np.divide(y, counts_max-counts_min)
print(X)

In [ ]:
print(y)

In [ ]:
# split into train and test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
y_train.shape

### Model

In [ ]:
# import dependencies
import keras
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.layers import Dense

In [ ]:
# custom loss function
def weighted_MSE(y_true, y_pred):
    # condition is true or false
    condition=tf.greater(y_true-y_pred, 0)
    # first option if true, second if false
    return tf.math.reduce_mean(tf.math.reduce_sum(tf.where(condition, (y_true-y_pred)**2, ((y_true-y_pred)**2)/16)))

In [ ]:
# create DNN model
dnn_model = keras.Sequential(
    [
        layers.Dense(6, activation="sigmoid", input_shape=(6,), name="layer1"),
        layers.Dense(6, activation="relu", name="layer2"),
        layers.Dense(1, activation="linear", name="layer3")
    ]
)
dnn_model.compile(optimizer='Adam', loss=weighted_MSE)
# train model and plot loss
model_2=dnn_model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the results
# group preds, actual counts, and observation number into one dataframe
y_pred=dnn_model.predict(X_test)
y_pred_df=pd.DataFrame(y_pred)
y_df=pd.concat([y_pred_df, pd.DataFrame(y_test)], axis=1)
y_df.columns=['preds', 'counts']

# unscale and take the exponential of the counts and preds
counts_column=y_df['counts'].tolist()
y_df['counts']=pd.Series([np.multiply(i, counts_max-counts_min) for i in counts_column])
counts_column=y_df['counts'].tolist()
y_df['counts']=pd.Series([round(np.around(np.exp(i)-1, 0), 0) for i in counts_column])
preds_column=y_df['preds'].tolist()
y_df['preds']=pd.Series([np.multiply(i, counts_max-counts_min) for i in preds_column])
preds_column=y_df['preds'].tolist()
y_df['preds']=pd.Series([round(np.around(np.exp(i)-1, 0), 0) for i in preds_column])
y_df=y_df.sort_values(by=['counts']).reset_index(drop=True)
y_df['wind-y component']=weather_complete[['windy009']]

In [ ]:
# R-squared score
ypred=y_df['preds'].tolist()
ytrue=y_df['counts'].tolist()

print(r2_score(ytrue,ypred))

In [ ]:
# plot predictions
xy=sns.scatterplot(x='wind-y component', y='value', hue='value type', style="value type", data=pd.melt(y_df, ['wind-y component'], var_name='value type', value_name='value'))
xy.set_ylim(0, 2000)

In [ ]:
from meteostat import Point, Daily, Hourly, Stations

In [ ]:
def get_weather(latitude, longitude, month, day, year):
    # find nearest station
    stations = Stations()
    stations = stations.nearby(latitude, longitude)
    station = stations.fetch(2)
    station=station.index
    defective_stations=['72506', '74492', '74494', 'KNZW0']
    # pick a non-defective station
    if station[0] in defective_stations:
        station = station[1]
    else:
        station = station[0]
    # hourly data for station
    weather_for_day=pd.DataFrame(index=[0], columns=['temp', 'prcp', 'wind_x', 'wind_y'])
    start = datetime(year, month, day, 0, 00)
    end = datetime(year, month, day, 12, 00)
    stn = Hourly(station, start, end)
    stn = stn.fetch()
    # fill missing rows of hourly weather with rows of NaN values
    times=[
        datetime(start.year, start.month, start.day, 0, 00),
        datetime(start.year, start.month, start.day, 1, 00),
        datetime(start.year, start.month, start.day, 2, 00),
        datetime(start.year, start.month, start.day, 3, 00), 
        datetime(start.year, start.month, start.day, 4, 00),
        datetime(start.year, start.month, start.day, 5, 00),
        datetime(start.year, start.month, start.day, 6, 00),
        datetime(start.year, start.month, start.day, 7, 00),
        datetime(start.year, start.month, start.day, 8, 00),
        datetime(start.year, start.month, start.day, 9, 00),
        datetime(start.year, start.month, start.day, 10, 00),
        datetime(start.year, start.month, start.day, 11, 00),
        datetime(start.year, start.month, start.day, 12, 00),
    ]
    nan_row=pd.Series({"temp":np.nan, "dwpt":np.nan, "rhum":np.nan, "prcp":np.nan, "snow": np.nan, "wdir":np.nan, "wspd":np.nan, "wpgt":np.nan, "pres":np.nan, "tsun":np.nan, "coco":np.nan})
    for time in times:
        if time not in stn.index:
            stn=pd.concat([stn, nan_row.to_frame(time).T])
    stn=stn.sort_index()
    # filter to every 3 hrs
    counter=-1 
    for index, row in stn.iterrows():
        counter+=1  
        if counter%3!=0:
            stn=stn.drop(index)
    # replace NaNs
    stn['prcp']=stn['prcp'].fillna(0)
    stn['temp']=stn['temp'].interpolate(method='linear', limit_direction='both')
    stn['wdir']=stn['wdir'].interpolate(method='linear', limit_direction='both')
    stn['wspd']=stn['wspd'].interpolate(method='linear', limit_direction='both')
    # create a list for each parameter
    stn_temp=stn['temp'].tolist()
    stn_prcp=stn['prcp'].tolist()
    stn_wdir=stn['wdir'].tolist()
    stn_wspd=stn['wspd'].tolist()
    # convert wind speed and wind direction into x and y components
    stn_wind_x=[]
    stn_wind_y=[]
    for i in range(5):
        cosine=np.around(np.cos(np.deg2rad(stn_wdir[i])), decimals=1)
        sine=np.around(np.sin(np.deg2rad(stn_wdir[i])), decimals=1)
        hourly_wind_x=np.around(float(stn_wspd[i])*cosine, decimals=1)
        hourly_wind_y=np.around(float(stn_wspd[i])*sine, decimals=1)
        stn_wind_x.append(hourly_wind_x)
        stn_wind_y.append(hourly_wind_y)
    stn_wind_x=pd.Series(stn_wind_x, dtype=object).fillna(0).tolist()
    stn_wind_y=pd.Series(stn_wind_y, dtype=object).fillna(0).tolist()
    # create a dataframe
    temp_df=pd.DataFrame([stn_temp], columns=['temp000','temp003','temp006','temp009','temp012'])
    prcp_df=pd.DataFrame([stn_prcp], columns=['prcp000','prcp003','prcp006','prcp009','prcp012'])
    wdir_df=pd.DataFrame([stn_wdir], columns=['wdir000','wdir003','wdir006','wdir009','wdir012'])
    wspd_df=pd.DataFrame([stn_wspd], columns=['wspd000','wspd003','wspd006','wspd009','wspd012'])
    windx_df=pd.DataFrame([stn_wind_x], columns=['windx000','windx003','windx006','windx009','windx012'])
    windy_df=pd.DataFrame([stn_wind_y], columns=['windy000','windy003','windy006','windy009','windy012'])
    weather_for_day=pd.concat([windx_df,windy_df], axis=1)
    # add day using day_in_the_year function
    def days_in_a_year(month, day):
        return np.piecewise(1, [month==1, month==2, month==3, month==4, month==5, month==6, month==7, month==8, month==9, month==10, month==11, month==12], [day,31+day,60+day,91+day,121+day,152+day,182+day,213+day,244+day,274+day,305+day,335+day])
    weather_for_day['days_cos']=np.cos(np.deg2rad(days_in_a_year(month, day)*(60/61)))
    weather_for_day['days_sin']=np.sin(np.deg2rad(days_in_a_year(month, day)*(60/61)))
    return weather_for_day

In [ ]:
def df_to_numpy_scaled(weather_for_day):
    # keep specific columns
    weather_0609_cos_sin_1=weather_for_day[weather_for_day.columns[2:4]]
    weather_0609_cos_sin_2=weather_for_day[weather_for_day.columns[7:9]]
    weather_0609_cos_sin_3=weather_for_day[weather_for_day.columns[-2:]]
    weather_0609_cos_sin=pd.concat([weather_0609_cos_sin_1,weather_0609_cos_sin_2, weather_0609_cos_sin_3], axis=1)
    # convert dataframe to numpy array
    x_test=weather_0609_cos_sin.to_numpy()
    # scale weather parameters, latitude, and longitude, so that the standard deviation is 1
    # normalize X values using StandardScaler() formula: z=(x-u)/s
    # wind_X
    x_test[0, 0:2]-=windx_avg
    x_test[0, 0:2]=np.divide(x_test[0, 0:2], windx_std)
    # wind_y
    x_test[0, 2:4]-=windy_avg
    x_test[0, 2:4]=np.divide(x_test[0, 2:4], windy_std)
    return x_test

In [ ]:
def predict_northern_gannets(latitude, longitude, month, day, year):
    # create x_test
    x_test=df_to_numpy_scaled(get_weather(latitude, longitude, month, day, year))
    # predict number of nogas
    noga_count=dnn_model.predict(x_test, verbose=0)
    # the noga count in scaled so we unscale it
    noga_count=float(str(noga_count).replace("[", "").replace("]", ""))
    # get original count by using the formula: y=z*(max-min)+min
    noga_count=np.multiply(noga_count, counts_max-counts_min)
    noga_count+=counts_min
    noga_count=round(np.exp(noga_count)-1,0)
    if 0<=noga_count<10:
        noga_prediction='\nDate: '+str(month)+'/'+str(day)+'/'+str(year)+'\nIn the morning, there are expected to be less than 10 Northern Gannets at this location.'
    if 10<=noga_count<50:
        noga_prediction='\nDate: '+str(month)+'/'+str(day)+'/'+str(year)+'\nIn the morning, there are expected to be 10-50 Northern Gannets at this location.'
    if 50<=noga_count<100:
        noga_prediction='\nDate: '+str(month)+'/'+str(day)+'/'+str(year)+'\nIn the morning, there are expected to be 50-100 Northern Gannets at this location.'
    if 100<=noga_count<250:
        noga_prediction='\nDate: '+str(month)+'/'+str(day)+'/'+str(year)+'\nIn the morning, there are expected to be 100-250 Northern Gannets at this location.'
    if 250<=noga_count<500:
        noga_prediction='\nDate: '+str(month)+'/'+str(day)+'/'+str(year)+'\nIn the morning, there are expected to be 250-500 Northern Gannets at this location.'
    if 500<=noga_count<=1000:
        noga_prediction='\nDate: '+str(month)+'/'+str(day)+'/'+str(year)+'\nIn the morning, there are expected to be 500-1000 Northern Gannets at this location.'
    if noga_count>1000:
        noga_prediction='\nDate: '+str(month)+'/'+str(day)+'/'+str(year)+'\nIn the morning, there are expected to be greater than 1000 Northern Gannets at this location.'
    return noga_prediction

In [ ]:
print(predict_northern_gannets(42.691015, -70.631701, 1, 13, 2024)) # 459, 'ESE, 30-45 mph'
print(predict_northern_gannets(42.691015, -70.631701, 1, 17, 2024)) # ~0-10, 'WNW, 10 mph'
print(predict_northern_gannets(42.691015, -70.631701, 5, 28, 2024)) # 823, 'S, 10-15 mph'
print(predict_northern_gannets(42.691015, -70.631701, 10, 2, 2022)) # 1500, 'E, 20-30 mph'
print(predict_northern_gannets(42.691015, -70.631701, 1, 28, 2024)) # 1.0, 'SE, 5-12 mph'
print(predict_northern_gannets(42.691015, -70.631701, 1, 10, 2024)) # 180, 'ENE, 5-10 mph'
print(predict_northern_gannets(42.691015, -70.631701, 2, 10, 2024)) # 0.0, 'WSW, 0-5 mph'
print(predict_northern_gannets(42.691015, -70.631701, 11, 1, 2023)) # 500, 'E, 5-10 mph'
print(predict_northern_gannets(42.691015, -70.631701, 9, 16, 2023)) # 475, 'NW, 30-35 mph'